<a href="https://colab.research.google.com/github/aivrm/ai/blob/main/EX_10_10_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%8B%A8%EC%96%B4_Level%EB%A1%9C_%EB%B2%88%EC%97%AD%EA%B8%B0_%EC%97%85%EA%B7%B8%EB%A0%88%EC%9D%B4%EB%93%9C%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EXPLORATION_BS2

---
# 10. 번역기를 만들어보자

---

---
## 10-10. 프로젝트 : 단어 Level로 번역기 업그레이드하기

---

프로젝트를 진행하기 전에 주요 라이브러리 버전을 확인해 보죠.

In [ ]:
import tensorflow

print(tensorflow.__version__)

2.8.2


실습에서 구현한 번역기는 글자 단위(Character-level)에서 구현된 번역기였습니다.<br>
하지만 실제 번역기의 경우에는 글자 단위가 아니라 단어 단위(Word-level)에서 구현되는 것이 좀 더 보편적입니다.<br>

동일한 데이터셋을 사용하면서 글자 단위와는 다른 전처리와 임베딩 층(Embedding layer)를 추가하여 단어 단위의 번역기를 완성시켜보겠습니다.<br>
하지만, 단어 단위로 할 경우에는 단어의 개수가 글자 단위로 했을 경우와 비교하여 단어장의 크기(Vocabulary) 크기도 커지고, 학습 속도도 좀 더 느려집니다.<br>
**학습과 테스트 시의 원활한 진행을 위해서 데이터에서 상위 33,000개의 샘플만 사용해주세요.**<br>

**33000개 중 3000개는 테스트 데이터로 분리하여 모델을 학습한 후에 번역을 테스트 하는 용도로 사용합니다.**

---
### Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)

---

글자 단위가 아닌 단어 단위의 번역기를 하기 위해서는 글자 단위에서는 신경쓰지 않았던 몇 가지 추가적인 전처리가 필요합니다.

#### 1. 구두점(Punctuation)을 단어와 분리해주세요.

일반적으로 영어권 언어의 경우에는 띄어쓰기 단위로 단어를 분리합니다.<br>
토큰화(Tokenization)이라고도 불리는 이 작업은 어디서부터 어디까지가 하나의 단어인지를 구분하는 작업인데요.<br>
그런데 띄어쓰기를 해주기 전에 구두점을 분리하는 작업이 필요할 때가 있습니다.<br>
<br>
예를 들어서 'he is a good boy!'라는 문장이 있을 때, 이를 띄어쓰기 단위로 토큰화한다면<br>
['he', 'is', 'a', 'good', 'boy!']가 됩니다.<br>
<br>
그런데 실제로 !는 boy와 붙어있는 한 단어가 아니므로<br>
좀 더 올바른 전처리는 ['he', 'is', 'a', 'good', 'boy', '!']가 맞습니다.<br>
<br>
!나 ? 또는 온점과 같은 특수문자들을 구두점(punctuation)이라고 부릅니다.<br>
이들을 토큰화하기 전에 단어와 미리 분리시켜주세요!<br>
<br>
분리 전 : he is a Good boy!<br>
분리 후 : he is a Good boy !

#### 2. 소문자로 바꿔주세요.

기계가 보기에는 스펠링이 같더라도 대문자로 된 단어와 소문자로 된 단어는 서로 다른 단어입니다.<br>
예를 들어 'Good'과 'good'은 기계가 보기에는 다른 단어입니다.<br>
그래서 모든 문장에 대해서 전부 영어로 바꿔주는 작업을 하겠습니다.<br>
<br>
변환 전 : he is a Good boy !<br>
변환 후 : he is a good boy !

#### 3. 띄어쓰기 단위로 토큰를 수행하세요.

띄어쓰기 단위로 토큰화를 수행해서 단어를 분리하는 작업을 해주세요.<br>
기계는 이렇게 분리된 토큰들을 각각 하나의 단어로 인식할 수 있게 됩니다.<br>
<br>
토큰화 전 : 'he is a good boy !'<br>
토큰화 후 : ['he', 'is', 'a', 'good', 'boy', '!']

In [1]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
# 19만개 중 33000개만 사용
num_samples = 33000

In [4]:
def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

# 전처리
def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [5]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


---
### Step 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어주세요.

---

글자 단위 번역기를 구현할 때와 마찬가지로 디코더의 입력 시퀀스 맨 앞에는 시작을 의미하는 토큰인 가 필요합니다. 그리고 교사 강요를 수행할 때, 디코더의 실제값이 되는 디코더의 레이블 시퀀스에는 종료를 의미하는 종료 토큰 가 필요합니다.<br>
<br>
예를 들어 번역 문장이 Courez!이었다고 한다면, Step 1을 거친 후에는 다음과 같은 결과를 얻습니다.<br>
<br>
Step 1을 수행한 후 : ['courez', '!']<br>
<br>
이 문장에 대해서 각각 디코더의 입력 시퀀스와 레이블 시퀀스를 만들면 다음과 같습니다.<br>
<br>
입력 시퀀스 : ['', 'courez', '!']<br>
레이블 시퀀스 : ['courez', '!', ']<br>
<br>
참고로 Step 2가 반드시 Step 1이 끝난 후에 이루어질 필요는 없습니다!<br>
<br>
Step 1을 수행하는 중간에 수행해도 상관없습니다.<br>

In [6]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

In [7]:
# 5개만 샘플로 보기
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>']]


---
### Step 3. 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요.

---

딥 러닝 모델은 각 단어를 텍스트가 아닌 숫자를 처리합니다.<br>
케라스 토크나이저를 사용해서 각 단어를 고유한 정수로 바꿔보세요.<br>
케라스 토크나이저의 사용법은<br>
아래의 링크에서 2. 케라스(Keras)의 텍스트 전처리 에 설명되어져 있습니다.<br>

---
▣ 위키독스<br>
https://wikidocs.net/31766

---

위 링크의 가이드를 통해서 영어와 프랑스어에 대한 토크나이저를 각각 생성하고,<br>
tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환해보세요.<br>

In [8]:
# keras 의 Tokenizer를 이용하여 처리
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [9]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (33000, 8)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


In [10]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4672, 프랑스어 단어 집합의 크기 : 8153


In [11]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [12]:
# Data shuffle
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [ 3275 15449 28895 ...  2165 14495 17123]


In [13]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [14]:
encoder_input[30997]

array([  15,    8,    9,  131, 1106,    1,    0,    0], dtype=int32)

In [16]:
decoder_input[30997]

array([  2,  19,   5,  24, 241, 837,   1,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [18]:
decoder_target[30997]

array([ 19,   5,  24, 241, 837,   1,   3,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

33000 개 중 10%인 3300개 를 테스트 데이터로 함.

In [19]:
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [20]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [21]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 8)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 8)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


---
### Step 4. 임베딩 층(Embedding layer) 사용하기

---

이번에는 입력이 되는 각 단어를 임베딩 층을 사용하여 벡터화하겠습니다.

임베딩 층을 사용하는 방법과 그 설명에 대해서는 아래의 링크의

1. 케라스 임베딩 층(Keras Embedding layer) 를 참고하세요.

---
▣ 위키독스<br>
https://wikidocs.net/33793

---

실제 번역기 구현을 위해서 사용할 수 있는 인코더 코드의 예시는 다음과 같습니다.

이를 통해서 인코더와 디코더의 임베딩 층을 각각 구현해보세요.

```python
from tensorflow.keras.layers import Input, mbedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(단어장의 크기, 임베딩 벡터의 차원)(encoder_inputs)
encoder_lstm = LSTM(hidden state의 크기, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
```

주의할 점은 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만,<br>
디코더의 훈련 과정과 테스트 과정(예측 과정)에서의 임베딩 층은 동일해야 합니다!

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [22]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [23]:
embedding_dim = 64
hidden_units = 64

In [25]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [26]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 21s 38ms/step - loss: 3.4153 - acc: 0.6116 - val_loss: 2.0704 - val_acc: 0.6162
Epoch 2/50
233/233 [==============================] - 6s 27ms/step - loss: 1.8730 - acc: 0.6788 - val_loss: 1.7596 - val_acc: 0.7375
Epoch 3/50
233/233 [==============================] - 6s 27ms/step - loss: 1.6700 - acc: 0.7420 - val_loss: 1.6227 - val_acc: 0.7509
Epoch 4/50
233/233 [==============================] - 6s 28ms/step - loss: 1.5444 - acc: 0.7558 - val_loss: 1.5087 - val_acc: 0.7602
Epoch 5/50
233/233 [==============================] - 6s 28ms/step - loss: 1.4383 - acc: 0.7652 - val_loss: 1.4232 - val_acc: 0.7699
Epoch 6/50
233/233 [==============================] - 6s 27ms/step - loss: 1.3508 - acc: 0.7798 - val_loss: 1.3417 - val_acc: 0.7880
Epoch 7/50
233/233 [==============================] - 6s 27ms/step - loss: 1.2682 - acc: 0.7956 - val_loss: 1.2741 - val_acc: 0.7995
Epoch 8/50
233/233 [==============================] - 6s 27ms/step -

---
### Step 5. 모델 구현하기
---

글자 단위 번역기에서 구현한 모델을 참고로 단어 단위 번역기의 모델을 완성시켜보세요!

In [27]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [28]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

---
### Step 6. 모델 평가하기

---

단어 단위 번역기에 대해서 훈련 데이터의 샘플과 테스트 데이터의 샘플에 대해서 번역 문장을 만들어보고 정답 문장과 번역 문장을 비교해보세요.

In [29]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [30]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : please step aside . 
정답문장 : veuillez vous mettre de cote . 
번역문장 : veuillez vous montrer . 
--------------------------------------------------
입력문장 : i m trapped . 
정답문장 : je suis piegee . 
번역문장 : je suis piege . 
--------------------------------------------------
입력문장 : you re precise . 
정답문장 : tu es precis . 
번역문장 : tu es precis . 
--------------------------------------------------
입력문장 : please be polite . 
정답문장 : veuillez vous montrer polies . 
번역문장 : veuillez vous montrer polie . 
--------------------------------------------------
입력문장 : we meant well . 
정답문장 : nos intentions etaient bonnes . 
번역문장 : nos intentions etaient bonnes . 
--------------------------------------------------


veuillez vous mettre de cote (옆에 서주세요)<br>
veuillez vous montrer (자신을 보여주세요)<br>
<br>
je suis piegee (내가 갇혀있어)<br>
je suis piege (덫에 걸렸어)<br>
<br>
tu es precis (당신은 정확합니다)<br>
<br>
veuillez vous montrer polies (예의를 지켜주세요)<br>
veuillez vous montrer polie (예의를 지켜주세요)<br>
<br>
nos intentions etaient bonnes (우리의 의도는 좋았어)<br>

In [31]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : that house is big . 
정답문장 : cette maison est grande . 
번역문장 : c est la maison . 
--------------------------------------------------
입력문장 : go home . 
정답문장 : rentre chez toi . 
번역문장 : va la maison ! 
--------------------------------------------------
입력문장 : we know everything . 
정답문장 : nous savons tout . 
번역문장 : nous savons en ville ! 
--------------------------------------------------
입력문장 : how tall you are ! 
정답문장 : que vous etes grandes ! 
번역문장 : comme vous etes grands ! 
--------------------------------------------------
입력문장 : i am so exhausted ! 
정답문장 : je suis si epuise ! 
번역문장 : je suis si idiot . 
--------------------------------------------------


cette maison est grande (이 집은 크다)<br>
c est la maison (그것은 집이다)<br>
<br>
rentre chez toi (집에가)<br>
va la maison (집에가)<br>
<br>
nous savons tout (우리는 모든 것을 알고 있습니다)<br>
nous savons en ville (우리는 마을에서 알고)<br>
<br>
que vous etes grandes 당신이 키가 크다는 것을)<br>
comme vous etes grands (당신이 얼마나 큰)<br>
<br>
je suis si epuise (너무 지쳤어)<br>
je suis si idiot (나는 정말 바보 야)<br>

---
### 회고

---

어렵고,<br>
신기하고,<br>
경이롭다.<br>
<br>
언어 쪽은 역시 나랑 안 맞는다는 것을 다시 느끼게 해주었으며,<br>
<br>
우선 보는 것을 먼저 습득하게 되면,<br>
언어적인 것으로 가야 하고,<br>
궁극적으로는 인간이 감각이나 사고를 뛰어넘는 영역으로 가야 하지 않을까 생각해 보았습니다.